# Проект по курсу ММОТ 2023

### Сидоров Леонид 617 ВМК МГУ

Темой моего проекта было `Обучение BERT классификации с информацией из базы знаний`, постановка задачи заключается в следующем

```
Найти датасет с текстами, содержащими большое число именованых сущностей 
(можно найти готовый или разметить сущности самому, на основе базы знаний).
Провести эксперименты:
оценить качество классификации на сырых документах
оценить качество классификации при одновременном обучении разметке и классификации текстов
оценить качество классификации, когда нужные слова в тексте помечаются "маркерами"
```

В поисках подходящего набора данных я решил немного переформулирвать задачу и свёл её к задаче NLU. В ней есть два таргета:

1. интент пользователя, то есть его основное намеренние (задача multi label классификации);
2. слоты, то есть важные смысловые части запроса, которые важны для реализации интента (задача тегирования последовательности, NER).

Выделять слоты можно различными способами, но было выбрано тегирование: исходные слоты переводятся в BIO-разметку, по которой решается задача NER.

Одно наблюдение имеет следующий вид

```JSON
{
    "text": "Yes, from 25 past 23 on",
    "intents": [
      "affirm"
    ],
    "slots": {
      "time_from": {
        "text": "25 past 23",
        "span": [
          10,
          20
        ],
        "value": {
          "hour": 23,
          "minute": 25
        }
      }
    }
  },
```

В общем случае интентов может быть больше одного, в нашем примере это `affirm`, то есть подтверждение. Слотов в одном тексте также может быть много, нас интересует не конкретное его значение, а тип и расположение по токенам. То есть слотовую разметку данного текста можно представить в виде

```
O O B-time_from I-time_from I-time_from O
```

Который говорит нам, что в конце предложения клиент говорил о времени, в которое он подтвердил встречу.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from transformers import logging

logging.set_verbosity_error()

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer, TrainingArguments, Trainer, \
    DataCollatorForTokenClassification, DataCollatorWithPadding, \
    AutoModelForTokenClassification, AutoModelForSequenceClassification

## Выгрузка набора данных

Рассматриваемый набор данных состоит из обращений пользователей в службу поддержки в двух разных сферах услуг: отелях и банках.

Данные было сложно размечать, поэтому его размер сильно ограничен, всего порядка 220 сложно структурированных наблюдений. Поэтому замерять качество мы будем с помощью усреднённой валидации на 20 фолдах, как было предложено авторами набора данных [здесь](https://github.com/PolyAI-LDN/task-specific-datasets/tree/master/nlupp). Выборка делится на 20 фолдов, после чего последовательно 2 подряд идущих фолда используется для валидации, а все остальные для обучения, итого получаем 10 наборов данных для обучения и валидации. Эти 2 фолда относятся к двум разным предметным областям.

Использование же больших предобученных моделей, таких как BERT, поможет нам обобщиться даже для такого небольшого набора наблюдений.

In [4]:
from data_loader import DataLoader

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer_params = dict(truncation=True)

loader = DataLoader("./data", tokenizer, tokenizer_params)
datasets = loader.get_data_for_experiment(domain="all", regime="large")
info_columns = ["text", "intents", "tagger", "tokens"]

## Раздельная классификация интентов и поиск слотов

В этом разделе мы решим две основные задачи NLU по отдельности, а именно сначала решим задачу NER, где в качестве сущностей будут выступать слоты в BIO разметке, а потом задачу многоклассовой классификации интентов, в которой одной фразе может быть сопоставлено сразу несколько намерений пользователя.

Для теггирования была выбрана `AutoModelForTokenClassification`, стандартная обёртка вокруг модели DistilBERT из библиотеки `HuggingFace`. Большая часть логирования была отключена, потому что при обучении 10 версий модели информации становится слишком много. Однако выключить вообще все выводы в HuggingFace оказалось нетривиальной задачей.

In [10]:
args = TrainingArguments("distilbert-finetuned",
                        learning_rate=1e-5,
                        optim="adamw_torch_fused",
                        per_device_train_batch_size=128,
                        per_device_eval_batch_size=128,
                        num_train_epochs=10,
                        weight_decay=0.01,
                        warmup_steps = 10,
                        lr_scheduler_type = 'linear',
                        evaluation_strategy="no",
                        #eval_steps=10,
                        seed=42,
                        fp16=True,
                        logging_strategy='no',
                        #logging_steps=200,
                        save_strategy="no",
                        disable_tqdm=True,
)

In [16]:
from metrics import ner_metrics

tagger_result = defaultdict(list)

for fold_dataset in datasets.values():
    model = AutoModelForTokenClassification.from_pretrained(
        "distilbert-base-uncased",
        id2label=loader.index2tag,
        label2id=loader.tag2index
    ).to('cuda')
    
    train_dataset = fold_dataset['train'].remove_columns(info_columns + ["classification_labels"]).rename_column("tagging_labels", "labels")
    test_dataset = fold_dataset['test'].remove_columns(info_columns + ["classification_labels"]).rename_column("tagging_labels", "labels")

    trainer = Trainer(model=model,
                    args=args,
                    train_dataset = train_dataset,
                    eval_dataset = test_dataset,
                    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
                    compute_metrics=ner_metrics,
                    tokenizer=tokenizer)

    trainer.train()
    for name, val in trainer.evaluate().items():
        if val != 'epoch':
            tagger_result[name].append(val)

tagger_result = pd.DataFrame(tagger_result)
tagger_result.mean()

{'train_runtime': 39.844, 'train_samples_per_second': 696.717, 'train_steps_per_second': 5.522, 'train_loss': 0.8640869834206321, 'epoch': 10.0}
{'eval_loss': 0.46589499711990356, 'eval_tagger_precision': 0.3181818181818182, 'eval_tagger_recall': 0.2962962962962963, 'eval_tagger_f1': 0.30684931506849317, 'eval_tagger_accuracy': 0.8907395069953364, 'eval_runtime': 0.1674, 'eval_samples_per_second': 1815.626, 'eval_steps_per_second': 17.917, 'epoch': 10.0}
{'train_runtime': 40.5591, 'train_samples_per_second': 683.694, 'train_steps_per_second': 5.424, 'train_loss': 0.8749995838512074, 'epoch': 10.0}
{'eval_loss': 0.3664020001888275, 'eval_tagger_precision': 0.41975308641975306, 'eval_tagger_recall': 0.37158469945355194, 'eval_tagger_f1': 0.39420289855072466, 'eval_tagger_accuracy': 0.914079648792725, 'eval_runtime': 0.182, 'eval_samples_per_second': 1686.768, 'eval_steps_per_second': 16.483, 'epoch': 10.0}
{'train_runtime': 40.0193, 'train_samples_per_second': 691.416, 'train_steps_per_s

eval_loss                     0.423975
eval_tagger_precision         0.401112
eval_tagger_recall            0.362711
eval_tagger_f1                0.380636
eval_tagger_accuracy          0.904554
eval_runtime                  0.197180
eval_samples_per_second    1580.782100
eval_steps_per_second        15.398600
epoch                        10.000000
dtype: float64

В табличке выше мы получили усреднённые значения метрик классификации токенов по всем 10 моделям. Полученные метрики пока мало что нам говорят (кроме того, что среднее качество весьма скудное), но мы будем использовать их в качестве бейзлайна при сравнении с другими подходами. Также мы можем заметить, что метрики на разных фолдах достаточно сильно различаются, в некоторых случаях расхождение может достигнуть 30%.

Задача классификации документов является для нас основной, поточу что именно она будет решаться на третем этапе этой работы, а именно использовании дополнительных признаков в модели в виде газетиров.

---

Газетиры (gazetteers) - это фактически добавление каких-то дополнительных признаков к токенам или их эмбедингам. Ясно, что при использовании BERT модификация эмбедингов сломает претрейн модели, поэтому в нашем случае стоит добавлять признаки уже к выходным эмбедингам берта, конкретные реализации будут рассмотрены в конце ноутбука.

---

Для теггирования была выбрана `AutoModelForSequenceClassification`, тоже стандартная обёртка вокруг модели DistilBERT из библиотеки `HuggingFace`.

In [18]:
args = TrainingArguments("distilbert-finetuned",
                        learning_rate=1e-5,
                        optim="adamw_torch_fused",
                        per_device_train_batch_size=128,
                        per_device_eval_batch_size=128,
                        num_train_epochs=15,            # больше эпох
                        weight_decay=0.01,
                        warmup_steps = 10,
                        lr_scheduler_type = 'linear',
                        evaluation_strategy="no",
                        #eval_steps=10,
                        seed=42,
                        fp16=True,
                        logging_strategy='no',
                        #logging_steps=200,
                        save_strategy="no",
                        disable_tqdm=True,
)

In [19]:
from metrics import multi_label_metrics

classifier_result = defaultdict(list)

for fold_dataset in datasets.values():
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", 
        problem_type="multi_label_classification", 
        num_labels=len(loader.index2intent.keys()), 
        id2label=loader.index2intent, 
        label2id=loader.intent2index
    ).to('cuda')
    
    train_dataset = fold_dataset['train'].remove_columns(info_columns + ["tagging_labels"]).rename_column("classification_labels", "labels")
    test_dataset = fold_dataset['test'].remove_columns(info_columns + ["tagging_labels"]).rename_column("classification_labels", "labels")

    trainer = Trainer(model=model,
                    args=args,
                    train_dataset = train_dataset,
                    eval_dataset = test_dataset,
                    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                    compute_metrics=multi_label_metrics,
                    tokenizer=tokenizer)

    trainer.train()
    for name, val in trainer.evaluate().items():
        if val != 'epoch':
            classifier_result[name].append(val)

classifier_result = pd.DataFrame(classifier_result)
classifier_result.mean()

{'train_runtime': 62.7008, 'train_samples_per_second': 664.106, 'train_steps_per_second': 5.263, 'train_loss': 0.37494384303237455, 'epoch': 15.0}
{'eval_loss': 0.2475365549325943, 'eval_classification_roc_auc': 0.626742329640797, 'eval_runtime': 0.1331, 'eval_samples_per_second': 2284.582, 'eval_steps_per_second': 22.545, 'epoch': 15.0}
{'train_runtime': 61.8479, 'train_samples_per_second': 672.537, 'train_steps_per_second': 5.336, 'train_loss': 0.3690942244096236, 'epoch': 15.0}
{'eval_loss': 0.24480561912059784, 'eval_classification_roc_auc': 0.5693521269117261, 'eval_runtime': 0.1388, 'eval_samples_per_second': 2211.863, 'eval_steps_per_second': 21.614, 'epoch': 15.0}
{'train_runtime': 61.2003, 'train_samples_per_second': 678.182, 'train_steps_per_second': 5.392, 'train_loss': 0.36905413540926846, 'epoch': 15.0}
{'eval_loss': 0.24309861660003662, 'eval_classification_roc_auc': 0.5787579269696219, 'eval_runtime': 0.1416, 'eval_samples_per_second': 2211.083, 'eval_steps_per_second': 

eval_loss                         0.244524
eval_classification_roc_auc       0.575151
eval_runtime                      0.148300
eval_samples_per_second        2091.036100
eval_steps_per_second            20.378100
epoch                            15.000000
dtype: float64

Для валидации задачи классификации интентов была выбрана только метрика AUC-ROC, потому что она не зависит от порога бинаризации в мульти лейбл классификации, и нам не нужно тратить драгоценные данные на подбор правильного порога для честного замера других метрик классификации.

На общем фоне выделяется первый фолд, на котором значение метрики особенно высоко и достигает 0.63, остальные модели достигают качества в районе 0.57.

## Одновременная классификация и поиск слотов

В этом разделе мы напишем свою модель в `HuggingFace` на основе DistilBERT для одновременной классификации интентов и выделения слотов. Фактически она является объединением стандартных классов из предыдущего раздела, то есть головы и функционалы ошибок для обеих задач одинаковы, но теперь они решаются в multi task режиме и с общим backbone.

In [36]:
from multitask_model import MultiTaskConfig, DistilBertForMultiTask, MultiTaskDataCollator

config = MultiTaskConfig.from_pretrained("distilbert-base-uncased", 
                                         num_labels_intents=len(loader.index2intent.keys()),
                                         index2intent=loader.index2intent,
                                         intent2index=loader.intent2index,
                                         num_labels_tags=len(loader.index2tag.keys()),
                                         index2tag=loader.index2tag,
                                         tag2index=loader.tag2index
    )

args = TrainingArguments("distilbert-finetuned",
                        learning_rate=1e-5,
                        optim="adamw_torch_fused",
                        per_device_train_batch_size=128,
                        per_device_eval_batch_size=128,
                        num_train_epochs=20,            # ещё больше эпох
                        weight_decay=0.01,
                        warmup_steps = 10,
                        lr_scheduler_type = 'linear',
                        evaluation_strategy="no",
                        #eval_steps=10,
                        seed=42,
                        fp16=True,
                        logging_strategy='no',
                        #logging_steps=200,
                        save_strategy="no",
                        disable_tqdm=True,
)

In [37]:
from metrics import multitask_metrics

multitask_result = defaultdict(list)

for fold_dataset in datasets.values():
    model = DistilBertForMultiTask.from_pretrained("distilbert-base-uncased", config=config).to('cuda')
    
    train_dataset = fold_dataset['train'].rename_column("classification_labels", "label_1").rename_column("tagging_labels", "label_2")
    test_dataset = fold_dataset['test'].rename_column("classification_labels", "label_1").rename_column("tagging_labels", "label_2")

    trainer = Trainer(model=model,
                    args=args,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset,
                    data_collator=MultiTaskDataCollator(tokenizer=tokenizer),
                    compute_metrics=multitask_metrics,
                    tokenizer=tokenizer)

    trainer.train()
    for name, val in trainer.evaluate().items():
        if val != 'epoch':
            multitask_result[name].append(val)

multitask_result = pd.DataFrame(multitask_result)
multitask_result.mean()

{'train_runtime': 83.0699, 'train_samples_per_second': 668.353, 'train_steps_per_second': 5.297, 'train_loss': 0.8417447176846591, 'epoch': 20.0}
{'eval_loss': 0.4513086974620819, 'eval_classification_roc_auc': 0.6658139434361856, 'eval_tagger_precision': 0.4694835680751174, 'eval_tagger_recall': 0.5291005291005291, 'eval_tagger_f1': 0.49751243781094523, 'eval_tagger_accuracy': 0.929713524317122, 'eval_runtime': 0.1831, 'eval_samples_per_second': 1660.215, 'eval_steps_per_second': 16.384, 'epoch': 20.0}
{'train_runtime': 82.2021, 'train_samples_per_second': 674.678, 'train_steps_per_second': 5.353, 'train_loss': 0.8589564930308949, 'epoch': 20.0}
{'eval_loss': 0.3858475983142853, 'eval_classification_roc_auc': 0.6449431381950002, 'eval_tagger_precision': 0.601063829787234, 'eval_tagger_recall': 0.6174863387978142, 'eval_tagger_f1': 0.6091644204851753, 'eval_tagger_accuracy': 0.9479460645970523, 'eval_runtime': 0.1887, 'eval_samples_per_second': 1626.564, 'eval_steps_per_second': 15.895

eval_loss                         0.419304
eval_classification_roc_auc       0.654235
eval_tagger_precision             0.552969
eval_tagger_recall                0.585682
eval_tagger_f1                    0.568681
eval_tagger_accuracy              0.939946
eval_runtime                      0.203820
eval_samples_per_second        1517.718400
eval_steps_per_second            14.787900
epoch                            20.000000
dtype: float64

Мы можем заметить, что решение в multi task режиме существенно увеличило качество для обеих задач. 

1. Для классификации AUC-ROC вырос с 0.58 до 0.65, причём значение метрики на разных фолдах теперь различается в рамках одного процентного пункта.
2. В теггировании F1-score тоже заметно вырос с 0.38 до 0.56 (что теперь выглядит приемлемо), но разброс значений на разных фолдах сохранился (есть разница в 20%).

## Классификация с дополнительными признаками

В этом разделе мы попробуем добавить дополнительные признаки к обрабатываемым словам. Теперь помимо самого текста на вход модели мы будем также подавать и BIO метки для теггирования в качестве признаков.

Конечно, такая модификация потребует от нас пересмотр всей модели (а точнее её головы). Часть, отвечающая за классификацию останется прежней, то есть будет состоять из двух линейных слоёв, разделённых ReLU и dropout. Но теперь ей будет предшествовать слой BiLSTM, которому на вход будет подаваться конкатенация газетиров и эмбедингов BERT. 

Выбор архитектуры обусловлен необходимостью: менять эмбединги на входе берта нельзя, это испортит претрейн модели. Остаётся только добавить новые признаки к выходным эмбедингам BERT, то есть отказаться от специального токена `[CLS]`, который выдаёт общий эмбеддинг документа, а вместо него заново агрегировать сконкатенированные эмбединги реккурентой моделью. В качестве этой реккурентой модели был выбран BiLSTM, как это часто делают в смежной литературе [1](https://arxiv.org/pdf/1511.08308.pdf) и [2](https://aclanthology.org/W19-5807.pdf).

LSTM оказалась очень нестабильной моделью, поэтому поначалу большая часть запусков просто расходилась в Nan или достигала плохого локального оптимума. Частично решить эту проблему помогли перенос скрытого состояния в следующий батч и хорошая инициализация весов модели через ортоганальные матрицы и метод Ксавьера.

Осталось только понять, как подавать газетиры в модель. Первым проверим лобовой подход, а именно перевод индексов тегов в обучаемые эмбеддинги.

In [45]:
from gazetteers_model import GazetteerConfig, DistilBertForGazetteer, GazetteerDataCollator

args = TrainingArguments("distilbert-finetuned",
                        learning_rate=1e-5,
                        optim="adamw_torch_fused",
                        per_device_train_batch_size=128,
                        per_device_eval_batch_size=128,
                        num_train_epochs=20,            # ещё больше эпох
                        weight_decay=0.01,
                        warmup_steps=10,
                        lr_scheduler_type='linear',
                        evaluation_strategy="no",
                        #eval_steps=10,
                        seed=42,
                        fp16=True,
                        logging_strategy='no',
                        #logging_steps=200,
                        save_strategy="no",
                        disable_tqdm=True,
)

In [46]:
config = GazetteerConfig.from_pretrained("distilbert-base-uncased", 
                                         num_labels_intents=len(loader.index2intent.keys()),
                                         index2intent=loader.index2intent,
                                         intent2index=loader.intent2index,
                                         num_labels_tags=len(loader.index2tag.keys()),
                                         index2tag=loader.index2tag,
                                         tag2index=loader.tag2index,
                                         batch_size=args.per_device_train_batch_size,
                                         use_gaz_features=True,
                                         use_gaz_embeds=True,
                                         gaz_embeds_dim=len(loader.index2tag.keys())+1
    )

In [47]:
gazetteer_result_embeds = defaultdict(list)

folds = list(datasets.values())
i = 0

while i < len(folds):
    try:
        model = DistilBertForGazetteer.from_pretrained("distilbert-base-uncased", config=config).to('cuda')

        train_dataset = folds[i]['train'].rename_column("classification_labels", "label").rename_column("tagging_labels", "gaz_features")
        test_dataset = folds[i]['test'].rename_column("classification_labels", "label").rename_column("tagging_labels", "gaz_features")

        trainer = Trainer(model=model,
                        args=args,
                        train_dataset=train_dataset,
                        eval_dataset=test_dataset,
                        data_collator=GazetteerDataCollator(num_labels_tags=len(loader.index2tag.keys()), is_embeds=config.use_gaz_embeds, tokenizer=tokenizer),
                        compute_metrics=multi_label_metrics,
                        tokenizer=tokenizer)

        trainer.train()
        for name, val in trainer.evaluate().items():
            if val != 'epoch':
                gazetteer_result_embeds[name].append(val)
        i += 1
    except ValueError:
        print("Nan output, bad try :(")
        

gazetteer_result_embeds = pd.DataFrame(gazetteer_result_embeds)
gazetteer_result_embeds.mean()

{'train_runtime': 99.3752, 'train_samples_per_second': 558.691, 'train_steps_per_second': 4.428, 'train_loss': 0.24322456013072621, 'epoch': 20.0}
{'eval_loss': 0.13645081222057343, 'eval_classification_roc_auc': 0.7330673745169303, 'eval_runtime': 0.1663, 'eval_samples_per_second': 1828.034, 'eval_steps_per_second': 18.04, 'epoch': 20.0}
{'train_runtime': 100.6947, 'train_samples_per_second': 550.774, 'train_steps_per_second': 4.37, 'train_loss': 0.2390063545920632, 'epoch': 20.0}
{'eval_loss': 0.13722047209739685, 'eval_classification_roc_auc': 0.7179127643859314, 'eval_runtime': 0.1835, 'eval_samples_per_second': 1672.663, 'eval_steps_per_second': 16.345, 'epoch': 20.0}
{'train_runtime': 86.1841, 'train_samples_per_second': 642.114, 'train_steps_per_second': 5.105, 'train_loss': 0.6923958518288352, 'epoch': 20.0}
Nan output, bad try :(
{'train_runtime': 86.8639, 'train_samples_per_second': 637.088, 'train_steps_per_second': 5.065, 'train_loss': 0.6923958518288352, 'epoch': 20.0}
Nan

eval_loss                         0.414412
eval_classification_roc_auc       0.602095
eval_runtime                      0.190630
eval_samples_per_second        1623.077800
eval_steps_per_second            15.818200
epoch                            20.000000
dtype: float64

В этой части мы замеряем только метрики классификации. AUC-ROC здесь скачет от 0.47 до 0.73 на разных фолдах, что говорит нам о недостатке данных для обучения подобных эмбеддингов, среднее качество проигрывает multi task подходу. Кроме того, здесь мы можем заметить ту самую численную нестабильность, когда модель выучивает Nan в качестве ответа.

В поисках лучшего подхода мы можеи прочитать статью [Improving Neural Named Entity Recognition with Gazetteers](https://arxiv.org/pdf/2003.03072.pdf), где авторы не выучивают эмбединги, а просто подают one-hot вектора на вход LSTM. На наших данных подавать one-hot вектора напрямую в рекуррентный слой не получилось, он также разваливался из-за большого количества нулей на входе. Поэтому сейчас мы протестируем решение, в котором эти вектора сначала обрабатываются линейным словем, а только потом идут на вход BiLSTM.

In [48]:
config = GazetteerConfig.from_pretrained("distilbert-base-uncased", 
                                         num_labels_intents=len(loader.index2intent.keys()),
                                         index2intent=loader.index2intent,
                                         intent2index=loader.intent2index,
                                         num_labels_tags=len(loader.index2tag.keys()),
                                         index2tag=loader.index2tag,
                                         tag2index=loader.tag2index,
                                         batch_size=args.per_device_train_batch_size,
                                         use_gaz_features=True,
                                         use_gaz_embeds=False,
                                         gaz_embeds_dim=len(loader.index2tag.keys())+1
    )

In [50]:
gazetteer_result_onehot = defaultdict(list)

folds = list(datasets.values())
i = 0

while i < len(folds):
    try:
        model = DistilBertForGazetteer.from_pretrained("distilbert-base-uncased", config=config).to('cuda')

        train_dataset = folds[i]['train'].rename_column("classification_labels", "label").rename_column("tagging_labels", "gaz_features")
        test_dataset = folds[i]['test'].rename_column("classification_labels", "label").rename_column("tagging_labels", "gaz_features")

        trainer = Trainer(model=model,
                        args=args,
                        train_dataset=train_dataset,
                        eval_dataset=test_dataset,
                        data_collator=GazetteerDataCollator(num_labels_tags=len(loader.index2tag.keys()), 
                                                            is_embeds=config.use_gaz_embeds, tokenizer=tokenizer),
                        compute_metrics=multi_label_metrics,
                        tokenizer=tokenizer)

        trainer.train()
        for name, val in trainer.evaluate().items():
            if val != 'epoch':
                gazetteer_result_onehot[name].append(val)
        i += 1
    except ValueError:
        print("Nan output, bad try :(")
        

gazetteer_result_onehot = pd.DataFrame(gazetteer_result_onehot)
gazetteer_result_onehot.mean()

{'train_runtime': 110.5529, 'train_samples_per_second': 502.203, 'train_steps_per_second': 3.98, 'train_loss': 0.23068960363214666, 'epoch': 20.0}
{'eval_loss': 0.13621975481510162, 'eval_classification_roc_auc': 0.7377128062235251, 'eval_runtime': 0.219, 'eval_samples_per_second': 1387.907, 'eval_steps_per_second': 13.696, 'epoch': 20.0}
{'train_runtime': 110.8735, 'train_samples_per_second': 500.21, 'train_steps_per_second': 3.968, 'train_loss': 0.23299232829700817, 'epoch': 20.0}
{'eval_loss': 0.13850541412830353, 'eval_classification_roc_auc': 0.7166078827420657, 'eval_runtime': 0.2424, 'eval_samples_per_second': 1266.576, 'eval_steps_per_second': 12.377, 'epoch': 20.0}
{'train_runtime': 109.1589, 'train_samples_per_second': 506.968, 'train_steps_per_second': 4.031, 'train_loss': 0.24163001667369496, 'epoch': 20.0}
{'eval_loss': 0.13349366188049316, 'eval_classification_roc_auc': 0.7200070818405728, 'eval_runtime': 0.2407, 'eval_samples_per_second': 1300.374, 'eval_steps_per_second

eval_loss                         0.135607
eval_classification_roc_auc       0.726332
eval_runtime                      0.245000
eval_samples_per_second        1260.690000
eval_steps_per_second            12.285700
epoch                            20.000000
dtype: float64

Это решение было лишено недостатков предшественника, поэтому оно выдаёт стабильное качество в районе 0.73 на всех фолдах. По сравнению с multi task подходом мы получили значимое улучшение AUC-ROC с 0.65 до 0.73. Но теперь давайте проверим, не связан ли наш прирост метрик с банальным добавлением дополнительного BiLSTM слоя в модель, и запустим ту же самую архитектуру, но без входа с газетирами.

In [51]:
config = GazetteerConfig.from_pretrained("distilbert-base-uncased", 
                                         num_labels_intents=len(loader.index2intent.keys()),
                                         index2intent=loader.index2intent,
                                         intent2index=loader.intent2index,
                                         num_labels_tags=len(loader.index2tag.keys()),
                                         index2tag=loader.index2tag,
                                         tag2index=loader.tag2index,
                                         batch_size=args.per_device_train_batch_size,
                                         use_gaz_features=False,
                                         use_gaz_embeds=False,
                                         gaz_embeds_dim=len(loader.index2tag.keys())+1
    )

In [52]:
gazetteer_result_clear = defaultdict(list)

folds = list(datasets.values())
i = 0

while i < len(folds):
    try:
        model = DistilBertForGazetteer.from_pretrained("distilbert-base-uncased", config=config).to('cuda')

        train_dataset = folds[i]['train'].rename_column("classification_labels", "label").rename_column("tagging_labels", "gaz_features")
        test_dataset = folds[i]['test'].rename_column("classification_labels", "label").rename_column("tagging_labels", "gaz_features")

        trainer = Trainer(model=model,
                        args=args,
                        train_dataset=train_dataset,
                        eval_dataset=test_dataset,
                        data_collator=GazetteerDataCollator(num_labels_tags=len(loader.index2tag.keys()), 
                                                            is_embeds=config.use_gaz_embeds, tokenizer=tokenizer),
                        compute_metrics=multi_label_metrics,
                        tokenizer=tokenizer)

        trainer.train()
        for name, val in trainer.evaluate().items():
            if val != 'epoch':
                gazetteer_result_clear[name].append(val)
        i += 1
    except ValueError:
        print("Nan output, bad try :(")
        

gazetteer_result_clear = pd.DataFrame(gazetteer_result_clear)
gazetteer_result_clear.mean()

{'train_runtime': 109.4688, 'train_samples_per_second': 507.176, 'train_steps_per_second': 4.019, 'train_loss': 0.23920405994762073, 'epoch': 20.0}
{'eval_loss': 0.13595330715179443, 'eval_classification_roc_auc': 0.7363404314485632, 'eval_runtime': 0.2158, 'eval_samples_per_second': 1408.525, 'eval_steps_per_second': 13.9, 'epoch': 20.0}
{'train_runtime': 109.6344, 'train_samples_per_second': 505.863, 'train_steps_per_second': 4.013, 'train_loss': 0.23795346346768467, 'epoch': 20.0}
{'eval_loss': 0.13685797154903412, 'eval_classification_roc_auc': 0.7190076841590349, 'eval_runtime': 0.2439, 'eval_samples_per_second': 1258.465, 'eval_steps_per_second': 12.298, 'epoch': 20.0}
{'train_runtime': 108.8358, 'train_samples_per_second': 508.472, 'train_steps_per_second': 4.043, 'train_loss': 0.23872347745028408, 'epoch': 20.0}
{'eval_loss': 0.13323919475078583, 'eval_classification_roc_auc': 0.7281413737487478, 'eval_runtime': 0.2335, 'eval_samples_per_second': 1340.288, 'eval_steps_per_secon

eval_loss                         0.135218
eval_classification_roc_auc       0.725065
eval_runtime                      0.239470
eval_samples_per_second        1293.762400
eval_steps_per_second            12.609000
epoch                            20.000000
dtype: float64

Как мы видим, качество осталось практически без изменений. Поэтому гипотеза о том, что газетиры значимо увеличивают показатели модели, не подтвердилась на наших данных.

## Выводы

А теперь давайте посмотрим на все полученные нами результаты. При изучении теггера стало ясно, что multitask задача в постановке NLU даёт значимый прирост в качестве теггера (что также верно и для классификации интентов). Для классификации multitask дополнительно снижает дисперсию оценки.

In [62]:
print(f"single tagger f1:                      {tagger_result['eval_tagger_f1'].mean().round(3)} +- {tagger_result['eval_tagger_f1'].std().round(3)}")
print(f"multitask tagger f1:                   {multitask_result['eval_tagger_f1'].mean().round(3)} +- {multitask_result['eval_tagger_f1'].std().round(3)}")

single tagger f1:                      0.381 +- 0.047
multitask tagger f1:                   0.569 +- 0.048


Газетиры же не предоставили нам прирост в качестве модели, дополнительные слои даже ухудшили стабильность работы всей архитектуры. Однако в ходе экспериментов мы поняли, что добавление BiLSTM на выход BERT модели является хорошим решением даже в задачах классификации документов.

In [63]:
print(f"single classifier roc-auc:             {classifier_result['eval_classification_roc_auc'].mean().round(3)} +- {classifier_result['eval_classification_roc_auc'].std().round(3)}")
print(f"multitask classifier roc-auc:          {multitask_result['eval_classification_roc_auc'].mean().round(3)} +- {multitask_result['eval_classification_roc_auc'].std().round(3)}")
print(f"gazetteer embeds classifier roc-auc:   {gazetteer_result_embeds['eval_classification_roc_auc'].mean().round(3)} +- {gazetteer_result_embeds['eval_classification_roc_auc'].std().round(3)}")
print(f"gazetteer onehot classifier roc-auc:   {gazetteer_result_onehot['eval_classification_roc_auc'].mean().round(3)} +- {gazetteer_result_onehot['eval_classification_roc_auc'].std().round(3)}")
print(f"gazetteer clear classifier roc-auc:    {gazetteer_result_clear['eval_classification_roc_auc'].mean().round(3)} +- {gazetteer_result_clear['eval_classification_roc_auc'].std().round(3)}")

single classifier roc-auc:             0.575 +- 0.019
multitask classifier roc-auc:          0.654 +- 0.008
gazetteer embeds classifier roc-auc:   0.602 +- 0.128
gazetteer onehot classifier roc-auc:   0.726 +- 0.008
gazetteer clear classifier roc-auc:    0.725 +- 0.008
